**Supported Models**
* BERT
* RoBERTa

## Load Dependencies

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from datasets import load_dataset
from transformers import DefaultDataCollator
from transformers import TrainingArguments, Trainer
from tqdm.auto import tqdm
import evaluate
import numpy as np
import collections
import os
from datetime import datetime

## Helper Functions

In [2]:
def newsqa2squad(sample):
    answers_text = sample["answers"]
    answers_start = sample["labels"][0]["start"]
    sample["answers"] = {'text':answers_text, 'answer_start':answers_start}
    return sample

def filter_function(examples):
    question = examples["question"]
    context = examples["context"]
    input_len = len(tokenizer(question,context, truncation=False).input_ids)
    return (input_len < tokenizer.model_max_length)

def tokenize(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        return_offsets_mapping=True,
        return_overflowing_tokens=True,
        padding="max_length",
    )
    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []
    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["id"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]
    inputs["example_id"] = example_ids
    return inputs

def compute_metrics(start_logits, end_logits, features, examples, metric):
    example_to_features = collections.defaultdict(list)
    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append(idx)
    
    n_best = 20
    max_answer_length = 30
    predicted_answers = []
    
    metric = evaluate.load("squad")
    
    for example in tqdm(examples):
        example_id = example["id"]
        context = example["context"]
        answers = []

        # Loop through all features associated with that example
        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Skip answers that are not fully in the context
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # Skip answers with a length that is either < 0 or > max_answer_length
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue

                    answer = {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                    answers.append(answer)

        # Select the answer with the best score
        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["logit_score"])
            predicted_answers.append(
                {"id": example_id, "prediction_text": best_answer["text"]}
            )
        else:
            predicted_answers.append({"id": example_id, "prediction_text": ""})

    theoretical_answers = [{"id": ex["id"], "answers": ex["answers"]} for ex in examples]
    return metric.compute(predictions=predicted_answers, references=theoretical_answers)

## Load Model and Tokenizer

In [3]:
seed = 42
num_proc = 4 # num of cpu workers
checkpoint = "varun-v-rao/bert-base-cased-squad"

torch.cuda.manual_seed(seed)
torch.manual_seed(seed)

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForQuestionAnswering.from_pretrained(checkpoint)

In [4]:
data_collator = DefaultDataCollator()

training_args = TrainingArguments(
    "./qa_tmp_outputs",
    per_device_eval_batch_size=32,
)

trainer = Trainer(
    model=model,
    #args=training_args,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

## Evaluate

In [6]:
num_test_epochs = 5

test_datasets = ['varun-v-rao/squad', 'lucadiliello/newsqa', 'varun-v-rao/adversarial_hotpotqa']
dataset2split = {'varun-v-rao/squad':"validation", 'lucadiliello/newsqa':"validation", 'varun-v-rao/adversarial_hotpotqa':"validation"}

metric = evaluate.load("squad")

for dataset_str in test_datasets:
    target_split = dataset2split[dataset_str]
    dataset = load_dataset(dataset_str, split=target_split)

    if dataset_str == 'lucadiliello/newsqa': 
        dataset = dataset.map(newsqa2squad, num_proc=num_proc).rename_column("key","id")
        dataset = dataset.filter(filter_function)

    tokenized_dataset = dataset.map(tokenize, batched=True, num_proc=num_proc, remove_columns=dataset.column_names)
    
    tmp_results = []
    print(f"--- Evaluating performance on {dataset_str} ---")
    for i in range(num_test_epochs):
        predictions, _, _ = trainer.predict(tokenized_dataset)
        start_logits, end_logits = predictions
        results = compute_metrics(start_logits, end_logits, tokenized_dataset, dataset, metric)
        tmp_results.append(results['f1'])
    
    print(f"Results array: {tmp_results}")   
    averaged_results = np.mean(np.asarray(tmp_results))
    print(f"Results averaged over {num_test_epochs} epochs: {averaged_results} %\n")

Map (num_proc=4):   0%|          | 0/10487 [00:00<?, ? examples/s]

--- Evaluating performance on varun-v-rao/squad ---


  0%|          | 0/10487 [00:00<?, ?it/s]

  0%|          | 0/10487 [00:00<?, ?it/s]

  0%|          | 0/10487 [00:00<?, ?it/s]

...

  0%|          | 0/2851 [00:00<?, ?it/s]

  0%|          | 0/2851 [00:00<?, ?it/s]

  0%|          | 0/2851 [00:00<?, ?it/s]

  0%|          | 0/2851 [00:00<?, ?it/s]

Results array: [34.22608470755477, 34.22608470755477, 34.22608470755477, 34.22608470755477, 34.22608470755477]
Results averaged over 5 epochs: 34.22608470755477 %

